# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
tf.__version__

'2.19.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
#Acess to GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

# Agora tens acesso ao teu Drive
# Exemplo: se puseste o dataset em "Meu Drive/datasets"

train_dir = "/content/drive/MyDrive/dataset/training_set"#Training directory
test_dir = "/content/drive/MyDrive/dataset/test_set"#Test directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#transformacoes às imgs de training set para evitar overfitting (IMG AUGMENTATION - depois verificámos que a accuracy em train set ficou 90% vs 99% (sem augmentation) mas depois no test ficou melhor com img augmentation. evitou o overfitting)
train_datagen = ImageDataGenerator(
        rescale=1./255, #aplica feature scaling aos pixeis (255 sao as cores)
        shear_range=0.2, #img augmentation (os três restantes)
        zoom_range=0.2,
        horizontal_flip=True)

#tornar as imagens mais faceis de digerir computacionalmente
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255) #scaling novamente
test_set = test_datagen.flow_from_directory(test_dir, #caminho para a pasta com o test_set
                                            target_size = (64, 64),
                                            batch_size = 32, #tamanho de batches
                                            class_mode = 'binary') #binary porque vai ser dog ou cat

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential() #instancia CNN

### Step 1 - Convolution

In [ ]:
#CONVOLUTION LAYER
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
#filters -> feature detectors para a convolution. Arquitetura com 32 costuma ser bom
#kernel_size -> tamanho da matriz feature detector. Como é =3 a matriz será 3x3 e ira correr a img com a lupa em 3x3
#input_shape -> shape das imagens. Como metemos as imagens anteriormente em 64x64, fica assim!

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [ ]:
#APLY POOLING (MAX POOLING. onde salta (slides) o tamanho todo da matriz em cada passo) ao feature map resultante do convolution layer
#ADD POOLING LAYER
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #tamanho da pool e strides 2 para ir saltando de 2 em 2 e correr 2x2.
#padding -> como lida qd apanha somente parte do feature map e fica com a matriz de análise fora. Neste caso nao faria diferença alterar, portanto x

### Adding a second convolutional layer

In [ ]:
#SEGUNDO LAYER CONVOLUTIONAL COM MAX POOLING
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) #input shape nao é preciso nesta porque ja temos uma layer de convolutional anterior
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
#FLATENNING
cnn.add(tf.keras.layers.Flatten()) # flats the results data into 1D vector

### Step 4 - Full Connection

In [ ]:
#ADD A NEW LAYER AND MAKE FULL CONNECTION
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #fully connected layer com 128 neuronios e funcao ativacao relu
#como temos imgs, convem ter mais neuronios. Daí metermos 128.
#ReLU sempre até ao output layer é o recomendado como função de ativação

### Step 5 - Output Layer

In [ ]:
#Terminar. Ligado ao anterior na mesma
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#Só vamos precisar de um neuronio pq a resposta é binária (cão ou gato) -> binary classification
#Activation -> Nao ReLU mas sim SIGMOID. Se fosse multiclass e não binário -> SOFTMAX


## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
#Compiling the CNN -> Optimizer (gradient descent -> adam), Loss functions (crossentropy error) and some metrics (accuracy)
#Binary classification -> So we'll do gradient descent to reduce the error
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
#TREINAR CNN
cnn.fit(x = training_set, validation_data = test_set, epochs = 10) #epochs convem ir experimentando e ver se a accuracy converge. Essa é a intenção

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5304 - loss: 0.6966

KeyboardInterrupt: 

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) #load image from google drive folder
#img em meter em 2D para o load.
test_image = image.img_to_array(test_image)
#adicionar questao do batch
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image) #vem em batch dimension.
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)